In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


In [24]:
# Загрузка данных
data = pd.read_csv('/home/eva/project/predictive_real_estate_price_analytics_service/data/progress/progress_data.csv')
data.sort_values(by='total_meters')

,url_id,total_meters,price
496,307723781,10.7,3200000
672,279298831,13.3,4287787
713,317285359,17.0,7890000
709,304863760,17.2,6141432
505,313373364,18.6,4270000
...,...,...,...
145,316935589,117.5,69500000
301,243257002,118.0,38700000
406,316491879,118.0,57700000
309,306278529,118.4,25900000


In [25]:

# Стандартизация признака
#scaler = StandardScaler()
#data['total_meters'] = scaler.fit_transform(data[['total_meters']])
'''
# Разделение на признаки и целевую переменную
X = data[['total_meters']]  # только один признак - площадь
y = data['price']
y_log = np.log1p(y)
y = y_log
# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
model = LinearRegression()
print(X_train)
print(y_train)
model.fit(X_train, y_train)


# Предсказание и обратное преобразование из логарифма
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_test_orig = np.expm1(y_test)

# Оценка модели
mse = mean_squared_error(y_test_orig, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_orig, y_pred)
mae = np.mean(np.abs(y_test_orig - y_pred))

# Вывод метрик
# Вывод метрик качества
# todo: use logging
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации R²: {r2:.6f}")
print(f"Средняя ошибка предсказания: {np.mean(np.abs(y_test - y_pred)):.2f} рублей")
'''





from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


X = data[['total_meters']]  # только один признак - площадь
y = data['price']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Создание и обучение модели CatBoost
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    verbose=100,  # выводит информацию каждые 100 итераций
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод метрик качества
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации R²: {r2:.6f}")
print(f"Средняя ошибка предсказания: {np.mean(np.abs(y_test - y_pred)):.2f} рублей")

# Для CatBoost коэффициенты модели интерпретируются иначе, чем в линейной регрессии
# Можно получить важность признаков
print("\nВажность признаков:")
for name, score in zip(X.columns, model.get_feature_importance()):
    print(f"{name}: {score:.2f}")


0:	learn: 22770405.9319198	total: 50ms	remaining: 50s
100:	learn: 15540025.4035100	total: 129ms	remaining: 1.15s
200:	learn: 15070105.2692003	total: 201ms	remaining: 799ms
300:	learn: 14672472.0298684	total: 291ms	remaining: 677ms
400:	learn: 14433027.8219812	total: 382ms	remaining: 571ms
500:	learn: 14201712.2612856	total: 468ms	remaining: 467ms
600:	learn: 14016205.4678744	total: 559ms	remaining: 371ms
700:	learn: 13886849.3498202	total: 636ms	remaining: 271ms
800:	learn: 13780933.5184039	total: 708ms	remaining: 176ms
900:	learn: 13693904.4489745	total: 788ms	remaining: 86.6ms
999:	learn: 13615863.8384430	total: 865ms	remaining: 0us
Среднеквадратичная ошибка (MSE): 286023944364415.69
Корень из среднеквадратичной ошибки (RMSE): 16912242.44
Коэффициент детерминации R²: 0.421292
Средняя ошибка предсказания: 12067201.36 рублей

Важность признаков:
total_meters: 100.00


In [26]:
import joblib

# Сохранение модели
model_path = '/home/eva/project/predictive_real_estate_price_analytics_service/models/linear_regression_model.pkl'

def model_pipline
joblib.dump(model, model_path)
print(f"Модель сохранена в файл {model_path}") 

# Загрузка модели
loaded_model = joblib.load(model_path)
print("Модель загружена из файла")

SyntaxError: expected '(' (1735167397.py, line 6)